## Results

In [5]:
import pandas as pd
import numpy as np
import sqlite3

TOTAL_FRAMES = 875_887
FPS = 30
SAMPLE_RATE = 10  # every 10th frame
WINDOW_SEC = 10
ALONE_DECISION_PERCENTAGE = 0.1  # at least 10% of frames in the window
GAP = 10 * FPS  # if segments are less than 5 seconds apart, merge them

face_frame_category_df = pd.read_csv("/home/nele_pauline_suffo/outputs/quantex_inference/face_frame_categories.csv")
person_frame_category_df = pd.read_csv("/home/nele_pauline_suffo/outputs/quantex_inference/person_frame_categories.csv")
person_face_combined_df = pd.read_csv("/home/nele_pauline_suffo/outputs/quantex_inference/combined_face_person_categories.csv")
alone_vs_not_alone_df = pd.read_csv("/home/nele_pauline_suffo/outputs/quantex_inference/alone_vs_not_alone.csv")

### Face Detection Information

In [33]:

# Calculate mutually exclusive counts
only_child = (face_frame_category_df['frame_category'] == 'only_child').sum()
only_adult = (face_frame_category_df['frame_category'] == 'only_adult').sum()
both_faces = (face_frame_category_df['frame_category'] == 'both_faces').sum()
no_faces = TOTAL_FRAMES - only_child - only_adult - both_faces

print(f"Frames with ONLY child faces: {only_child:,} ({only_child / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with ONLY adult faces: {only_adult:,} ({only_adult / TOTAL_FRAMES * 100:.2f}%)")  
print(f"Frames with BOTH face types: {both_faces:,} ({both_faces / TOTAL_FRAMES * 100:.2f}%)")
print(f"Frames with NO faces: {no_faces:,} ({no_faces / TOTAL_FRAMES * 100:.2f}%)")
print(f"Total frames analyzed: {TOTAL_FRAMES:,}")

Frames with ONLY child faces: 80,846 (9.23%)
Frames with ONLY adult faces: 66,720 (7.62%)
Frames with BOTH face types: 24,117 (2.75%)
Frames with NO faces: 704,204 (80.40%)
Total frames analyzed: 875,887


### Person Classification

In [34]:
# Calculate the numbers
only_child = person_frame_category_df['only_child_person'].iloc[0]
only_adult = person_frame_category_df['only_adult_person'].iloc[0] 
both_persons = person_frame_category_df['both_persons'].iloc[0]
no_persons = person_frame_category_df['no_persons'].iloc[0]
total_categorized = only_child + only_adult + both_persons + no_persons

print(f"Frames with ONLY child persons: {only_child:,} ({only_child / total_categorized * 100:.2f}%)")
print(f"Frames with ONLY adult persons: {only_adult:,} ({only_adult / total_categorized * 100:.2f}%)")
print(f"Frames with BOTH person types: {both_persons:,} ({both_persons / total_categorized * 100:.2f}%)")
print(f"Frames with NO persons: {no_persons:,} ({no_persons / total_categorized * 100:.2f}%)")
print(f"Total frames analyzed: {total_categorized:,}")

Frames with ONLY child persons: 2,279 (0.26%)
Frames with ONLY adult persons: 9,590 (1.09%)
Frames with BOTH person types: 1,031 (0.12%)
Frames with NO persons: 862,987 (98.53%)
Total frames analyzed: 875,887


### Combined Face and Person Presence Analysis

In [35]:
# Extract results
only_child = person_face_combined_df['only_child_present'].iloc[0]
only_adult = person_face_combined_df['only_adult_present'].iloc[0]
both_present = person_face_combined_df['both_present'].iloc[0]
no_one_present = person_face_combined_df['no_one_present'].iloc[0]
total_frames_analyzed = person_face_combined_df['total_frames_analyzed'].iloc[0]

print(f"Frames with ONLY child present: {only_child:,} ({only_child / total_frames_analyzed * 100:.2f}%)")
print(f"Frames with ONLY adult present: {only_adult:,} ({only_adult / total_frames_analyzed * 100:.2f}%)")
print(f"Frames with BOTH present: {both_present:,} ({both_present / total_frames_analyzed * 100:.2f}%)")
print(f"Frames with NO ONE present: {no_one_present:,} ({no_one_present / total_frames_analyzed * 100:.2f}%)")
print(f"Total frames analyzed: {total_frames_analyzed:,}")

# Additional insights
any_presence = only_child + only_adult + both_present
print(f"\nSummary Insights:")
print(f"Frames with ANY human presence: {any_presence:,} ({any_presence / total_frames_analyzed * 100:.2f}%)")
print(f"Frames with child presence: {only_child + both_present:,} ({(only_child + both_present) / total_frames_analyzed * 100:.2f}%)")
print(f"Frames with adult presence: {only_adult + both_present:,} ({(only_adult + both_present) / total_frames_analyzed * 100:.2f}%)")

Frames with ONLY child present: 81,819 (9.34%)
Frames with ONLY adult present: 73,470 (8.39%)
Frames with BOTH present: 25,973 (2.97%)
Frames with NO ONE present: 694,625 (79.31%)
Total frames analyzed: 875,887

Summary Insights:
Frames with ANY human presence: 181,262 (20.69%)
Frames with child presence: 107,792 (12.31%)
Frames with adult presence: 99,443 (11.35%)


#### Alone vs. Not Alone

In [2]:
WINDOW_SIZE_FRAMES = WINDOW_SEC * FPS #(e.g., 30s * 30fps = 90e0 frames)
WINDOW_SIZE_SAMPLED = WINDOW_SIZE_FRAMES // SAMPLE_RATE #(e.g., 900 // 10 = 90 frames)
THRESHOLD = WINDOW_SIZE_SAMPLED * ALONE_DECISION_PERCENTAGE #(e.g., 90 * 0.1 = 9 frames)

all_segments = []

for video_id, video_df in alone_vs_not_alone_df.groupby('video_id'):
    video_df = video_df.sort_values('frame_number').reset_index(drop=True)
    frame_numbers = video_df['frame_number'].to_numpy()
    not_alone = video_df['not_alone_detection'].to_numpy()
    
    # Sliding window: vectorized approach
    window_sums = np.convolve(not_alone, np.ones(WINDOW_SIZE_SAMPLED, dtype=int), mode='valid')
    valid_idxs = np.where(window_sums >= THRESHOLD)[0]
    
    if len(valid_idxs) == 0:
        continue

    # Create raw segments
    starts = frame_numbers[valid_idxs]
    ends = frame_numbers[valid_idxs + WINDOW_SIZE_SAMPLED - 1]
    segments = pd.DataFrame({'video_id': video_id, 'start_frame': starts, 'end_frame': ends})

    # Merge overlapping segments efficiently
    merged_segments = []
    current_start, current_end = segments.iloc[0][['start_frame', 'end_frame']]

    for s, e in segments[['start_frame', 'end_frame']].itertuples(index=False):
        if s <= current_end + GAP:
            current_end = max(current_end, e)
        else:
            merged_segments.append([video_id, current_start, current_end])
            current_start, current_end = s, e

    merged_segments.append([video_id, current_start, current_end])
    all_segments.extend(merged_segments)

# Convert to DataFrame
merged_df = pd.DataFrame(all_segments, columns=['video_id', 'start_frame', 'end_frame'])

# Convert frames → seconds
merged_df['start_time_sec'] = merged_df['start_frame'] / FPS
merged_df['end_time_sec'] = merged_df['end_frame'] / FPS

# Add video names
with sqlite3.connect('/home/nele_pauline_suffo/outputs/quantex_inference/interaction_inference.db') as conn:
    video_names = pd.read_sql("SELECT video_id, video_name FROM Videos", conn)

merged_df = merged_df.merge(video_names, on='video_id', how='left')

# Rearrange columns
merged_df = merged_df[['video_id', 'video_name', 'start_frame', 'end_frame', 'start_time_sec', 'end_time_sec']]

In [6]:
merged_df[merged_df['video_id'] == 10]

,video_id,video_name,start_frame,end_frame,start_time_sec,end_time_sec
69,10,quantex_at_home_id255237_2022_05_08_02,0,2050,0.000000,68.333333
70,10,quantex_at_home_id255237_2022_05_08_02,2600,3280,86.666667,109.333333
71,10,quantex_at_home_id255237_2022_05_08_02,7850,8560,261.666667,285.333333
72,10,quantex_at_home_id255237_2022_05_08_02,10110,10680,337.000000,356.000000
73,10,quantex_at_home_id255237_2022_05_08_02,13890,14580,463.000000,486.000000
74,10,quantex_at_home_id255237_2022_05_08_02,15970,16810,532.333333,560.333333
75,10,quantex_at_home_id255237_2022_05_08_02,28140,30630,938.000000,1021.000000
76,10,quantex_at_home_id255237_2022_05_08_02,31860,33120,1062.000000,1104.000000
77,10,quantex_at_home_id255237_2022_05_08_02,36300,37870,1210.000000,1262.333333
78,10,quantex_at_home_id255237_2022_05_08_02,38380,42100,1279.333333,1403.333333


In [4]:
merged_df[merged_df['video_id'] == 10]

,video_id,video_name,start_frame,end_frame,start_time_sec,end_time_sec
69,10,quantex_at_home_id255237_2022_05_08_02,0,2050,0.000000,68.333333
70,10,quantex_at_home_id255237_2022_05_08_02,2600,3280,86.666667,109.333333
71,10,quantex_at_home_id255237_2022_05_08_02,7850,8560,261.666667,285.333333
72,10,quantex_at_home_id255237_2022_05_08_02,10110,10680,337.000000,356.000000
73,10,quantex_at_home_id255237_2022_05_08_02,13890,14580,463.000000,486.000000
74,10,quantex_at_home_id255237_2022_05_08_02,15970,16810,532.333333,560.333333
75,10,quantex_at_home_id255237_2022_05_08_02,28140,30630,938.000000,1021.000000
76,10,quantex_at_home_id255237_2022_05_08_02,31860,33120,1062.000000,1104.000000
77,10,quantex_at_home_id255237_2022_05_08_02,36300,37870,1210.000000,1262.333333
78,10,quantex_at_home_id255237_2022_05_08_02,38380,42100,1279.333333,1403.333333


In [1]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = '/home/nele_pauline_suffo/ProcessedData/quantex_annotations/annotations.db'
interaction_conn = sqlite3.connect(db_path)

# Define the video IDs you want to filter
video_ids = [5, 6, 7, 8, 11, 23]

# Query annotations with selected columns
query_annotations = f"""
SELECT video_id, image_id, bbox, person_age
FROM annotations
WHERE video_id IN ({', '.join(map(str, video_ids))}) 
AND category_id = 10
AND outside = 0
"""

# Query videos table for file names
query_videos = f"""
SELECT id, file_name
FROM videos
WHERE id IN ({', '.join(map(str, video_ids))})
"""

# Load data into DataFrames
face_detections_gt = pd.read_sql(query_annotations, interaction_conn)
videos_df = pd.read_sql(query_videos, interaction_conn)

# Close the DB connection
interaction_conn.close()

# Merge the two DataFrames on video_id
gt_df = face_detections_gt.merge(videos_df, left_on="video_id", right_on="id", how="left")

# Create combined image_id: file_name + "_" + zero-padded image_id
gt_df["image_id"] = gt_df.apply(lambda row: f"{row['file_name'].replace('.mp4','')}_{int(row['image_id']):06d}", axis=1)

# map person_age infant to child
gt_df["person_age"] = gt_df["person_age"].replace({"infant": "child"})

# Drop the extra 'id' column if not needed
gt_df.drop(columns=["id", "file_name", "video_id"], inplace=True)
gt_df.rename(columns={"bbox": "bbox_gt", "person_age": "person_age_gt"}, inplace=True)

gt_df["gt_idx"] = gt_df.groupby("image_id").cumcount() + 1

gt_df_wide = gt_df.pivot(index="image_id", columns="gt_idx")
gt_df_wide.columns = [f"{col[0]}_{col[1]}" for col in gt_df_wide.columns]
gt_df_wide.reset_index(inplace=True)

print(gt_df_wide.head())

                                        image_id  \
0  quantex_at_home_id255944_2022_03_08_01_000030   
1  quantex_at_home_id255944_2022_03_08_01_000060   
2  quantex_at_home_id255944_2022_03_08_01_000090   
3  quantex_at_home_id255944_2022_03_08_01_000120   
4  quantex_at_home_id255944_2022_03_08_01_000360   

                            bbox_gt_1 bbox_gt_2 bbox_gt_3 person_age_gt_1  \
0   [1911.28, 500.33, 2304.0, 1079.9]       NaN       NaN           adult   
1     [1297.97, 20.78, 1736.4, 575.0]       NaN       NaN           adult   
2     [1927.6, 190.42, 2304.0, 780.5]       NaN       NaN           adult   
3      [2045.0, 66.45, 2304.0, 565.2]       NaN       NaN           adult   
4  [1097.67, 496.01, 1222.22, 645.69]       NaN       NaN           adult   

  person_age_gt_2 person_age_gt_3  
0             NaN             NaN  
1             NaN             NaN  
2             NaN             NaN  
3             NaN             NaN  
4             NaN             NaN  


In [9]:
import json
import pandas as pd
from datetime import datetime

# Load JSON
with open("/home/nele_pauline_suffo/outputs/face_detections/yolo12l_validation_20250822_112418/predictions.json", "r") as f:
    data = json.load(f)

# Convert to DataFrame
df_pred = pd.DataFrame(data)

# Extract video_id and frame_number from image_id
def parse_video_id(image_id):
    # example: quantex_at_home_id255944_2022_03_08_01_000000 return 000000 without trailing zeros
    parts = image_id.split("_")
    # video_id from idXXXXX
    vid = int(parts[3].replace("id", ""))  # "id255944" → 255944
    return vid

def parse_frame_number(image_id):
    # last part is frame number
    return int(image_id.split("_")[-1])

df_pred["frame_number"] = df_pred["image_id"].apply(parse_frame_number)
# filter to only keep predictions with score greater than 0.25
df_pred = df_pred[df_pred["score"] >= 0.25]

# Rename columns
df_pred.rename(columns={"score": "confidence_score", "category_id": "person_age_pred", "bbox": "bbox_pred"}, inplace=True)

# Add extra columns
df_pred["person_age_pred"] = df_pred["person_age_pred"].replace({1: "child", 2: "adult"})
# Reorder columns
final_columns = ["image_id", "bbox_pred", "person_age_pred"]
df_pred = df_pred[final_columns]

df_pred["gt_idx"] = df_pred.groupby("image_id").cumcount() + 1

df_pred_wide = df_pred.pivot(index="image_id", columns="gt_idx")
df_pred_wide.columns = [f"{col[0]}_{col[1]}" for col in df_pred_wide.columns]
df_pred_wide.reset_index(inplace=True)

print(df_pred_wide.head())

                                        image_id  \
0  quantex_at_home_id255944_2022_03_08_01_000030   
1  quantex_at_home_id255944_2022_03_08_01_000060   
2  quantex_at_home_id255944_2022_03_08_01_000090   
3  quantex_at_home_id255944_2022_03_08_01_000120   
4  quantex_at_home_id255944_2022_03_08_01_000360   

                             bbox_pred_1 bbox_pred_2 bbox_pred_3 bbox_pred_4  \
0   [1928.706, 534.306, 375.294, 547.87]         NaN         NaN         NaN   
1   [1292.824, 33.548, 434.423, 543.073]         NaN         NaN         NaN   
2  [1929.726, 192.167, 372.645, 562.045]         NaN         NaN         NaN   
3   [2050.465, 64.385, 253.136, 498.399]         NaN         NaN         NaN   
4   [1113.687, 496.063, 109.06, 156.085]         NaN         NaN         NaN   

  person_age_pred_1 person_age_pred_2 person_age_pred_3 person_age_pred_4  
0             adult               NaN               NaN               NaN  
1             adult               NaN               Na

In [10]:
# join both tables on image_id (outer join)
df_final = df_pred_wide.merge(gt_df_wide, on="image_id", how="outer")

In [11]:
df_final[df_final["bbox_gt_1"].isna()]

,image_id,bbox_pred_1,bbox_pred_2,bbox_pred_3,bbox_pred_4,person_age_pred_1,person_age_pred_2,person_age_pred_3,person_age_pred_4,bbox_gt_1,bbox_gt_2,bbox_gt_3,person_age_gt_1,person_age_gt_2,person_age_gt_3
13,quantex_at_home_id255944_2022_03_08_01_002010,"[2100.492, 1188.071, 86.245, 105.115]",NaN,NaN,NaN,child,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,quantex_at_home_id255944_2022_03_08_01_003690,"[1991.526, 0.33, 125.2, 123.789]",NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,quantex_at_home_id255944_2022_03_08_01_011220,"[127.639, 286.176, 71.385, 75.641]",NaN,NaN,NaN,child,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,quantex_at_home_id255944_2022_03_08_01_011460,"[115.714, 0.0, 1091.078, 425.623]",NaN,NaN,NaN,child,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,quantex_at_home_id255944_2022_03_08_01_011850,"[2002.212, 0.0, 301.549, 530.796]",NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,quantex_at_home_id260275_2022_04_28_01_008310,"[2251.084, 1105.558, 52.916, 172.597]",NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
673,quantex_at_home_id260275_2022_04_28_01_009270,"[2248.686, 101.383, 55.314, 189.439]",NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675,quantex_at_home_id260275_2022_04_28_01_009630,"[2086.594, 0.0, 217.204, 161.988]",NaN,NaN,NaN,child,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,quantex_at_home_id260275_2022_04_28_01_010290,"[396.361, 0.0, 745.816, 467.368]",NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
missclassified_df = df_final[
    (df_final["person_age_pred_1"] != df_final["person_age_gt_1"]) &
    df_final["person_age_pred_1"].notna() &
    df_final["person_age_gt_1"].notna() &
    df_final["person_age_gt_2"].isna()
]

In [ ]:
missclassified_df[["image_id", "person_age_pred_1", "persovn_age_gt_1"]]

,image_id,person_age_pred_1,person_age_gt_1
39,quantex_at_home_id255944_2022_03_08_01_003630,adult,child
65,quantex_at_home_id255944_2022_03_08_01_014160,adult,child
109,quantex_at_home_id255944_2022_03_10_01_009660,adult,child
622,quantex_at_home_id260275_2022_04_28_01_001140,child,adult
625,quantex_at_home_id260275_2022_04_28_01_001290,child,adult
629,quantex_at_home_id260275_2022_04_28_01_001560,child,adult
634,quantex_at_home_id260275_2022_04_28_01_001830,child,adult
639,quantex_at_home_id260275_2022_04_28_01_002280,child,adult
640,quantex_at_home_id260275_2022_04_28_01_002340,child,adult
644,quantex_at_home_id260275_2022_04_28_01_002550,child,adult


In [ ]:
frames_with_detections_query = """
WITH FramesWithDetections AS (
    -- First get unique frames with detections for each class and video
    SELECT
        v.id,
        v.video_id,
        d.frame_number,
        MAX(CASE WHEN d.object_class = 1.0 THEN 1 ELSE 0 END) as has_adult,
        MAX(CASE WHEN d.object_class = 0.0 THEN 1 ELSE 0 END) as has_child,
        MAX(CASE WHEN d.object_class = 3.0 THEN 1 ELSE 0 END) as has_adult_face,
        MAX(CASE WHEN d.object_class = 2.0 THEN 1 ELSE 0 END) as has_child_face,
        MAX(CASE WHEN d.object_class = 5.0 THEN 1 ELSE 0 END) as has_book,
        MAX(CASE WHEN d.object_class = 6.0 THEN 1 ELSE 0 END) as has_toy,
        MAX(CASE WHEN d.object_class = 7.0 THEN 1 ELSE 0 END) as has_kitchenware,
        MAX(CASE WHEN d.object_class = 8.0 THEN 1 ELSE 0 END) as has_screen,
        MAX(CASE WHEN d.object_class = 9.0 THEN 1 ELSE 0 END) as has_food,
        MAX(CASE WHEN d.object_class = 10.0 THEN 1 ELSE 0 END) as has_other_object
    FROM Videos v
    JOIN Detections d ON v.video_id = d.video_id
    WHERE v.video_id NOT IN (116, 13, 120, 45, 99, 427, 321, 387)
    GROUP BY v.id, v.video_id, d.frame_number
),
TotalFramesPerChild AS (
    -- Calculate total frames per child ID
    SELECT 
        v.id,
        SUM(vs.total_frames) as total_frames
    FROM Videos v
    JOIN VideoStatistics vs ON v.video_id = vs.video_id
    WHERE v.video_id NOT IN (116, 13, 120, 45, 99, 427, 321, 387)
    GROUP BY v.id
)
-- Then aggregate detections by child ID
SELECT 
    f.id,
    COUNT(DISTINCT f.video_id) as video_count,
    t.total_frames,
    SUM(f.has_adult) as frames_with_adult,
    SUM(f.has_child) as frames_with_child,
    SUM(f.has_adult_face) as frames_with_adult_face,
    SUM(f.has_child_face) as frames_with_child_face,
    SUM(f.has_book) as frames_with_book,
    SUM(f.has_toy) as frames_with_toy,
    SUM(f.has_kitchenware) as frames_with_kitchenware,
    SUM(f.has_screen) as frames_with_screen,
    SUM(f.has_food) as frames_with_food,
    SUM(f.has_other_object) as frames_with_other_object
FROM FramesWithDetections f
JOIN TotalFramesPerChild t ON f.id = t.id
JOIN VideoStatistics vs ON f.video_id = vs.video_id
WHERE vs.video_id NOT IN (116, 13, 120, 45, 99, 427, 321, 387)
GROUP BY f.id, t.total_frames
ORDER BY f.id;
"""

# Execute query and create DataFrame
frames_with_detections = pd.read_sql(frames_with_detections_query, conn)


# Calculate percentages
for col in frames_with_detections.columns[3:]:  # Skip id, video_count, and total_frames
    frames_with_detections[f'{col}_percent'] = (
        frames_with_detections[col] / frames_with_detections['total_frames'] * 100
    ).round(2)

# Display results
display(frames_with_detections)

,id,video_count,total_frames,frames_with_adult,frames_with_child,frames_with_adult_face,frames_with_child_face,frames_with_book,frames_with_toy,frames_with_kitchenware,...,frames_with_adult_percent,frames_with_child_percent,frames_with_adult_face_percent,frames_with_child_face_percent,frames_with_book_percent,frames_with_toy_percent,frames_with_kitchenware_percent,frames_with_screen_percent,frames_with_food_percent,frames_with_other_object_percent
0,254922,5,201729,4095,2756,2682,1479,688,1373,97,...,2.03,1.37,1.33,0.73,0.34,0.68,0.05,0.01,0.01,0.75
1,255237,6,282993,5405,15208,1570,6359,1165,7373,50,...,1.91,5.37,0.55,2.25,0.41,2.61,0.02,0.02,0.12,1.05
2,255695,4,68763,2165,2357,780,1008,5,881,630,...,3.15,3.43,1.13,1.47,0.01,1.28,0.92,0.00,0.00,0.47
3,255706,2,72922,3693,2122,1178,1359,379,1557,458,...,5.06,2.91,1.62,1.86,0.52,2.14,0.63,0.00,0.00,0.25
4,255944,5,138006,3224,4767,849,2661,2067,1244,244,...,2.34,3.45,0.62,1.93,1.50,0.90,0.18,0.00,0.03,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,271611,7,286457,2972,2895,2210,232,1005,3994,471,...,1.04,1.01,0.77,0.08,0.35,1.39,0.16,0.01,0.06,0.96
70,271700,6,200637,9361,2115,5093,1233,736,1244,130,...,4.67,1.05,2.54,0.61,0.37,0.62,0.06,0.02,0.04,1.06
71,272475,4,171070,5154,1768,1888,868,1211,676,169,...,3.01,1.03,1.10,0.51,0.71,0.40,0.10,0.01,0.02,1.17
72,273855,6,303755,6960,1994,2302,656,588,1060,74,...,2.29,0.66,0.76,0.22,0.19,0.35,0.02,0.01,0.01,0.47


In [23]:
# load csv into dataframe
df = pd.read_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/frames_with_detections.csv')
# add column processed_frames as total_frames / 10 and round up (show without decimals)
df['processed_frames'] = np.ceil(df['total_frames'] / 10).astype(int)

# make column processed_frames appear after total_frames
cols = df.columns.tolist()
cols.insert(3, cols.pop(cols.index('processed_frames')))
df = df.reindex(columns=cols)

# calculate percentage of frames with adults and all other classes, show percentage with two decimals
df['frames_with_adult_percent'] = (df['frames_with_adult'] / df['processed_frames'] * 100).round(2)
df['frames_with_child_percent'] = (df['frames_with_child'] / df['processed_frames'] * 100).round(2)
df['frames_with_adult_face_percent'] = (df['frames_with_adult_face'] / df['processed_frames'] * 100).round(2)
df['frames_with_child_face_percent'] = (df['frames_with_child_face'] / df['processed_frames'] * 100).round(2)
df['frames_with_book_percent'] = (df['frames_with_book'] / df['processed_frames'] * 100).round(2)
df['frames_with_toy_percent'] = (df['frames_with_toy'] / df['processed_frames'] * 100).round(2)
df['frames_with_kitchenware_percent'] = (df['frames_with_kitchenware'] / df['processed_frames'] * 100).round(2)
df['frames_with_screen_percent'] = (df['frames_with_screen'] / df['processed_frames'] * 100).round(2)
df['frames_with_food_percent'] = (df['frames_with_food'] / df['processed_frames'] * 100).round(2)
df['frames_with_other_object_percent'] = (df['frames_with_other_object'] / df['processed_frames'] * 100).round(2)

# calculate video_length from total_frames, video is taken at 30 fps
# divide by 30 then by 60 to get minutes
df['video_length_min'] = (df['total_frames'] / 30 / 60).round(2)
cols = df.columns.tolist()
cols.insert(3, cols.pop(cols.index('video_length_min')))
frames_with_detections_new = df.reindex(columns=cols)
frames_with_detections_new

,id,video_count,total_frames,video_length_min,processed_frames,video_length,frames_with_adult,frames_with_child,frames_with_adult_face,frames_with_child_face,...,frames_with_adult_percent,frames_with_child_percent,frames_with_adult_face_percent,frames_with_child_face_percent,frames_with_book_percent,frames_with_toy_percent,frames_with_kitchenware_percent,frames_with_screen_percent,frames_with_food_percent,frames_with_other_object_percent
0,254922,5,201729,112.07,20173,112.07,4095,2756,2682,1479,...,20.30,13.66,13.29,7.33,3.41,6.81,0.48,0.07,0.09,7.53
1,255237,6,282993,157.22,28300,157.22,5405,15208,1570,6359,...,19.10,53.74,5.55,22.47,4.12,26.05,0.18,0.19,1.18,10.46
2,255695,4,68763,38.20,6877,38.20,2165,2357,780,1008,...,31.48,34.27,11.34,14.66,0.07,12.81,9.16,0.01,0.00,4.74
3,255706,2,72922,40.51,7293,40.51,3693,2122,1178,1359,...,50.64,29.10,16.15,18.63,5.20,21.35,6.28,0.03,0.04,2.54
4,255944,5,138006,76.67,13801,76.67,3224,4767,849,2661,...,23.36,34.54,6.15,19.28,14.98,9.01,1.77,0.01,0.27,9.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,271611,7,286457,159.14,28646,159.14,2972,2895,2210,232,...,10.37,10.11,7.71,0.81,3.51,13.94,1.64,0.07,0.55,9.62
70,271700,6,200637,111.46,20064,111.46,9361,2115,5093,1233,...,46.66,10.54,25.38,6.15,3.67,6.20,0.65,0.19,0.44,10.62
71,272475,4,171070,95.04,17107,95.04,5154,1768,1888,868,...,30.13,10.33,11.04,5.07,7.08,3.95,0.99,0.15,0.15,11.70
72,273855,6,303755,168.75,30376,168.75,6960,1994,2302,656,...,22.91,6.56,7.58,2.16,1.94,3.49,0.24,0.11,0.07,4.72
